In [ ]:
"""
Import danych z pliku tekstowego
"""
    
def loadData():
    columns=[]
    
    data = [i.strip('\n').split(',') for i in open('')]
    for i in range(len(data[-1])):
        column="a"+str(i+1)
        columns.append(column)


    return data, columns


In [ ]:
from math import log
"""
Obliczenie entropii zdefiniowanego atrybutu
"""
def calcEntropy(data, column):
    #column - atrybut dla którego będzie obliczana entropia (-1 : ostatni(decyzyjny))
    entropy=0.0

    values = [data[column] for data in data] #wartosci wystepujace w atrybucie
    counter = dict([(i, values.count(i)) for i in values]) #slownik przechowujacy liczbe wystepujacych wartosci w tym atrybucie
    for key in counter:     
        entropy -= (counter[key]/len(data)) * log(counter[key]/len(data), 2) 
                                                           #len(data)-liczba wystepujacych wartości w atrybucie
    return entropy


In [ ]:
"""
Wybór atrybutu według którego będzie dokonany podział
Wybierany jest atrybut z najwyższym gainratio
"""

def chooseAttr(data):

    subdata=[] #zmienna przechowująca podtabelę danej wartości atrybutu z wyciętym sprawdzanym atrybutem
    attrlength = len(data[0])-1 #liczba atrybutów (bez decyzyjnego)
    baseentropy = calcEntropy(data, -1) #obliczenie entropii dla całej tabeli
    bestgainratio=0.0
    bestattr = -1 #zmienna przechowująca numer najlepszego atrybutu do podziału

    for i in range(attrlength):
        attrlist = [n[i] for n in data] #wypisanie wartosci atrybutu  w tablicy
        unique = set(attrlist) #przypisanie wartości atrybutu do zbioru(tylko unikalne wartości)
        newentropy = 0.0

        for value in unique:
            for row in data:
                if row[i] == value:  #jeśli w danym rowu i atrybucie wystepuje przekazana wartość z unique
                    reducedrow = row[:i] #zredukowany row tylko do kolumn poprzedzajacych sprawdzany atrybut
                    reducedrow.extend(row[i + 1:])#zredukowany row jedynie o wartość sprawdzanego atrybutu
                    subdata.append(reducedrow)
            newentropy += (len(subdata) / float(len(data))) * calcEntropy(subdata, -1) #obliczenie informacji(entropii) dla danego atrybutu
            subdata=[] #wyczyszczenie subdata 
        #print("Entropia atrybutu", column[i], newentropy)
        informationgain = baseentropy - newentropy #przyrost informacji dla danego atrybutu
        #print('Information Gain atrybutu', column[i], informationgain)
        splitinfo = calcEntropy(data, i) #split info dla danego atrybutu
        #print("Splitinfo atrybutu", column[i],  splitinfo)
        try:
            gainratio = informationgain/splitinfo #wartość zrównoważonego przyrostu informacji
        except:
            gainratio=0
        #print("Gainratio atrybutu", column[i], gainratio)
        #print("------------")
        if(gainratio >= bestgainratio):
            bestgainratio = gainratio
            bestattr = i
            
    #print('Numer atrybutu o najlepszym dopasowaniu:', bestattr)
    
    return bestattr


In [ ]:
"""
Główna funkcja algorytmu, która rekurencyjnie tworzy słownik zawierający drzewo decyzyjne
"""
def buildTree(data, column):

    subdata=[]
    values = [n[-1] for n in data] #wartosci ostatniego atrybutu

    if len(set(values))==1: #jezeli w decyzyjnej komumnie została tylko 1 wartość, zwróć tę wartość
        return values[0]
               
    bestattr2 = chooseAttr(data) #numer atrybutu z najwiekszym wspolczynnikiem gainratio
    bestattrcolumn = column[bestattr2] #nazwa atrybutu z najwiekszym wspolczynnikiem gainratio
    subcolumns = column[:] #etykiety kolumn   
    del(subcolumns[bestattr2]) #usunięcie etykiety kolumny wybranej do podziału    
    decisiontree = {bestattrcolumn: {}} #nazwa atrybutu z najwiekszym wspolczynnikien gainratio(dict)
    bestattrvalues = [n[bestattr2] for n in data] #wartości w atrybucie z najwiekszym wspolczynnikiem gainratio
    unique = set(bestattrvalues) ##przypisanie wartości atrybutu do zbioru(tylko unikalne wartości)
    #rekurencyjne budowanie kolejnych poziomów             
    for value in unique:
        for row in data:
            if row[bestattr2] == value:
                reducedrow = row[:bestattr2] #row zawierający wartości atrybutów znajdujących się przed wybranym atrybutem do podziału
                reducedrow.extend(row[bestattr2 + 1:])
                subdata.append(reducedrow)
        decisiontree[bestattrcolumn][value] = buildTree(subdata, subcolumns) #rekurencyjne budowanie drzewa
        subdata=[]

    return decisiontree


In [ ]:
import pydot
import uuid
"""
Generowanie obrazka png przedstawiającego drzewo decyzyjne
Źródło: https://stackoverflow.com/questions/24657384/plotting-a-decision-tree-with-pydot
"""

def generate_unique_node():
    """ Generate a unique node label."""
    return str(uuid.uuid1())

#liście
def create_node(graph, label, shape='circle'):
    node = pydot.Node(str(uuid.uuid1()), label=label, shape=shape,  style="bold")
    graph.add_node(node)
    return node

#linie
def create_edge(graph, node_parent, node_child, label):
    link = pydot.Edge(node_parent, node_child, label=label, style="dotted")
    graph.add_edge(link)
    return link

def walk_tree(graph, dictionary, prev_node=None):
    """ Recursive construction of a decision tree stored as a dictionary """
    for parent, child in dictionary.items():
        # root
        if not prev_node: 
            root = create_node(graph, parent)
            walk_tree(graph, child, root)
            continue
            
        # node
        if isinstance(child, dict):
            for p, c in child.items():
                n = create_node(graph, p)
                create_edge(graph, prev_node, n, str(parent))
                walk_tree(graph, c, n)
    
        # liście
        else: 
            leaf = create_node(graph, str(child), shape='box')
            create_edge(graph, prev_node, leaf, str(parent))

def plot_tree(dictionary, filename="output.png"):
    graph = pydot.Dot('drzewo', graph_type='graph', bgcolor="white")
    walk_tree(graph, tree)
    graph.write_png(filename)
    
    #my_networkx_graph = networkx.drawing.nx_pydot.from_pydot(graph)
    
data, column = loadData()
tree=buildTree(data, column)
print(tree)
plot_tree(tree)


{'a6': {'no': {'a5': {'15-17': 'no', '3-5': {'a4': {'30-34': {'a2': {'50-59': 'no', '40-49': 'no', '30-39': 'yes'}}, '20-24': {'a3': {'ge40': 'yes', 'premeno': 'no'}}, '40-44': 'yes', '25-29': {'a9': {'right_up': 'no', 'left_up': 'yes'}}, '10-14': 'no'}}, '12-14': 'yes', '9-11': 'yes', '0-2': {'a4': {'0-4': 'no', '35-39': 'no', '5-9': {'a9': {'central': 'no', 'left_low': 'yes', 'right_low': 'no', 'right_up': 'no'}}, '30-34': {'a1': {'no-recurrence-events': {'a7': {'2': {'a9': {'left_low': {'a8': {'left': {'a3': {'ge40': {'a2': {'60-69': {'a10': {'no': 'no', 'yes': 'yes'}}}}}}}}, 'right_low': 'no', 'right_up': 'yes', 'left_up': {'a2': {'40-49': 'yes', '30-39': 'no', '60-69': 'no'}}}}, '3': 'no', '1': 'no'}}, 'recurrence-events': {'a9': {'central': 'no', 'right_up': 'no', '?': 'no', 'left_up': {'a7': {'2': 'no', '3': 'yes', '1': 'no'}}, 'left_low': 'yes'}}}}, '15-19': {'a9': {'central': 'no', 'right_up': 'no', 'right_low': 'no', 'left_up': {'a3': {'ge40': {'a8': {'right': {'a7': {'3': 'y

In [ ]:
import json

"""
Generowanie drzewa w formie tekstowej z wcięciami
    
"""

rysDrzewo = json.dumps(tree, indent=4)
rysDrzewo = rysDrzewo.replace('"', "")
rysDrzewo = rysDrzewo.replace(',', "")
rysDrzewo = rysDrzewo.replace("{", "")
rysDrzewo = rysDrzewo.replace("}", "")
rysDrzewo = rysDrzewo.replace("\n    ", "\n")
rysDrzewo = rysDrzewo.replace("    ", " | ")

print(rysDrzewo)



a6: 
 | no: 
 |  | a5: 
 |  |  | 15-17: no
 |  |  | 3-5: 
 |  |  |  | a4: 
 |  |  |  |  | 30-34: 
 |  |  |  |  |  | a2: 
 |  |  |  |  |  |  | 50-59: no
 |  |  |  |  |  |  | 40-49: no
 |  |  |  |  |  |  | 30-39: yes
 |  |  |  |  |  | 
 |  |  |  |  | 
 |  |  |  |  | 20-24: 
 |  |  |  |  |  | a3: 
 |  |  |  |  |  |  | ge40: yes
 |  |  |  |  |  |  | premeno: no
 |  |  |  |  |  | 
 |  |  |  |  | 
 |  |  |  |  | 40-44: yes
 |  |  |  |  | 25-29: 
 |  |  |  |  |  | a9: 
 |  |  |  |  |  |  | right_up: no
 |  |  |  |  |  |  | left_up: yes
 |  |  |  |  |  | 
 |  |  |  |  | 
 |  |  |  |  | 10-14: no
 |  |  |  | 
 |  |  | 
 |  |  | 12-14: yes
 |  |  | 9-11: yes
 |  |  | 0-2: 
 |  |  |  | a4: 
 |  |  |  |  | 0-4: no
 |  |  |  |  | 35-39: no
 |  |  |  |  | 5-9: 
 |  |  |  |  |  | a9: 
 |  |  |  |  |  |  | central: no
 |  |  |  |  |  |  | left_low: yes
 |  |  |  |  |  |  | right_low: no
 |  |  |  |  |  |  | right_up: no
 |  |  |  |  |  | 
 |  |  |  |  | 
 |  |  |  |  | 30-34: 
 |  |  |  |  |  | a1: 
